In [1]:
import tensorflow as tf
tf.__version__

/Users/mzielinski/anaconda3/envs/keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/mzielinski/anaconda3/envs/keras/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.5.0'

### Load data

In [12]:
import pandas as pd
train_csv = pd.read_csv("/Users/mzielinski/.kaggle/competitions/titanic/train.csv",
                        names = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
                        skipinitialspace=True, low_memory=False, 
                        skiprows=1, na_values=[])

train_csv.fillna({'PassengerId': '', 'Survived': 0, 'Pclass': 0, 'Name': '', 'Sex': '', 'Age': 0, 'SibSp': 0,
       'Parch': 0, 'Ticket': '', 'Fare': 0, 'Cabin': '', 'Embarked': ''}, inplace=True)

labels = train_csv["Survived"]
feature_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
features = train_csv[feature_cols]

In [13]:
numeric_age = tf.feature_column.numeric_column("Age", default_value=features["Age"].mean())
numeric_fare = tf.feature_column.numeric_column("Fare", default_value=features["Fare"].mean())
numeric_features = [numeric_age, numeric_fare]

bucketized_age = tf.feature_column.bucketized_column(numeric_age, [0, 10, 20, 30, 40, 50, 60, 100])
one_hot_age = tf.feature_column.indicator_column(bucketized_age)
categorical_identity_cols = ["Pclass", "SibSp", "Parch"]
categorical_identity_features = [
    tf.feature_column.categorical_column_with_identity(
        key,
        len(features[key].unique()) + 1,
        0
    ) for key in categorical_identity_cols] 
one_hot_identity_features = [
    tf.feature_column.indicator_column(key) for key in categorical_identity_features]
categorical_dictionary_cols = ["Sex", "Embarked"]
categorical_dictionary_features = [
    tf.feature_column.categorical_column_with_vocabulary_list(
        key,
        list(features[key].dropna().unique())
    ) for key in categorical_dictionary_cols] 
one_hot_dictionary_features = [
    tf.feature_column.indicator_column(key) for key in categorical_dictionary_features]
categorical_crossed_age_sex = tf.feature_column.crossed_column(
    [bucketized_age, categorical_dictionary_features[0]],
    5000)
categorical_embedding_crossed_age_sex = tf.feature_column.embedding_column(categorical_crossed_age_sex, 9)
categorical_features = one_hot_identity_features + one_hot_dictionary_features + [one_hot_age, categorical_embedding_crossed_age_sex]
all_features = numeric_features + categorical_features

In [4]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

### Creating Custom Estimators

As the following figure shows, pre-made Estimators are **subclasses** of the `tf.estimator.Estimator` base class, while custom Estimators are an **instance** of `tf.estimator.Estimator`:

<img src="https://www.tensorflow.org/images/custom_estimators/estimator_types.png" width=600>

A **model function** (or `model_fn`) implements the ML algorithm. The only difference between working with pre-made Estimators and custom Estimators is with custom `Estimators`, you must write the model function. The model function we'll use has the following call signature:

    def my_model_fn(
       features, # This is batch_features from input_fn
       labels,   # This is batch_labels from input_fn
       mode,     # An instance of tf.estimator.ModeKeys
       params):  # Additional configuration

The caller may pass `params` to an Estimator's constructor. Any params passed to the constructor are in turn passed on to the `model_fn`.

### Define the model

#### Define the input layer

The first line of the model_fn calls `tf.feature_column.input_layer` to convert the feature dictionary and `feature_columns` into input for your model, as follows. The line applies the transformations defined by your feature columns, creating the model's input layer.

In [18]:
params = {"feature_columns": all_features, "hidden_units": [10, 10], "n_classes": 2}
net = tf.feature_column.input_layer(dict(features), params['feature_columns'])

#### Hidden layers

The `Layers API` provides a rich set of functions to define all types of hidden layers, including convolutional, pooling, and dropout layers. Here we're simply going to call `tf.layers.dense` to create hidden layers, with dimensions defined by `params['hidden_layers']`.

Note that `tf.layers.dense` provides many additional capabilities, including the ability to set a multitude of regularization parameters. For the sake of simplicity, though, we're going to simply accept the default values of the other parameters.

In [19]:
for units in params['hidden_units']:
    net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

#### Output layer

We'll define the output layer by calling `tf.layers.dense` yet again, this time without an activation function:

In [20]:
# Compute logits (1 per class).
logits = tf.layers.dense(net, params['n_classes'], activation=None)

### Implement training, evaluation, and prediction
The model function gets invoked whenever someone calls the Estimator's `train`, `evaluate`, or `predict` methods. Recall that the signature for the model function looks like this:

    def my_model_fn(
       features, # This is batch_features from input_fn
       labels,   # This is batch_labels from input_fn
       mode,     # An instance of tf.estimator.ModeKeys
       params):  # Additional configuration

Focus on that third argument, `mode`. As the following table shows, when someone calls train, evaluate, or predict, the Estimator framework invokes your model function with the mode parameter set as follows:

    |Estimator method | Estimator Mode   |
    |-----------------|------------------|
    |train()	      | ModeKeys.TRAIN   |
    |evaluate()       | ModeKeys.EVAL    |
    |predict()        | ModeKeys.PREDICT |

#### Predict

When the Estimator's `predict` method is called, the `model_fn` receives `mode = ModeKeys.PREDICT`. In this case, the model function must return a `tf.estimator.EstimatorSpec` containing the prediction. 

The prediction dictionary contains everything that your model returns when run in prediction mode. We return that dictionary to the caller via the `predictions` parameter of the `tf.estimator.EstimatorSpec`. The Estimator's `predict` method will yield these dictionaries.

In [24]:
mode = tf.estimator.ModeKeys.PREDICT

# Compute predictions.
predicted_classes = tf.argmax(logits, 1)
if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {
        'class_ids': predicted_classes[:, tf.newaxis],
        'probabilities': tf.nn.softmax(logits),
        'logits': logits,
    }
    out = tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
out

EstimatorSpec(mode='infer', predictions={'probabilities': <tf.Tensor 'Softmax:0' shape=(891, 2) dtype=float32>, 'class_ids': <tf.Tensor 'strided_slice:0' shape=(891, 1) dtype=int64>, 'logits': <tf.Tensor 'dense_8/BiasAdd:0' shape=(891, 2) dtype=float32>}, loss=None, train_op=None, eval_metric_ops={}, export_outputs=None, training_chief_hooks=(), training_hooks=(), scaffold=<tensorflow.python.training.monitored_session.Scaffold object at 0x120e51a90>, evaluation_hooks=())

#### Calculate the loss

For both training and evaluation we need to calculate the model's loss. This is the objective that will be optimized. 
We can calculate the loss by calling `tf.losses.sparse_softmax_cross_entropy`. 

In [25]:
# Compute loss.
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

#### Evaluate
When the Estimator's `evaluate` method is called, the `model_fn` receives `mode = ModeKeys.EVAL`. In this case, the model function must return a `tf.estimator.EstimatorSpec` containing the model's loss and optionally one or more metrics.

Although returning metrics is optional, most custom Estimators do return at least one metric. TensorFlow provides a Metrics module `tf.metrics` to calculate common metrics. The `tf.metrics.accuracy` function compares our predictions against the true values.

In [26]:
# Compute evaluation metrics.
accuracy = tf.metrics.accuracy(labels=labels,
                               predictions=predicted_classes,
                               name='acc_op')

The `EstimatorSpec` returned for evaluation typically contains the following information:

* **loss**, which is the model's loss
* **eval_metric_ops**, which is an optional dictionary of metrics.

In [28]:
metrics = {'accuracy': accuracy}
tf.summary.scalar('accuracy', accuracy[1])

if mode == tf.estimator.ModeKeys.EVAL:
    out = tf.estimator.EstimatorSpec(
        mode, loss=loss, eval_metric_ops=metrics)

#### Train
When the Estimator's `train` method is called, the `model_fn` is called with `mode = ModeKeys.TRAIN`. In this case, the model function must return an `EstimatorSpec` that contains the loss and a training operation.

Building the training operation will require an optimizer. We will use `tf.train.AdagradOptimizer` because we're mimicking the `DNNClassifier`, which also uses `Adagrad` by default. The `tf.train` package provides many other optimizers—feel free to experiment with them.

The `minimize` method also takes a `global_step` parameter. TensorFlow uses this parameter to count the number of training steps that have been processed (to know when to end a training run). Furthermore, the global_step is essential for TensorBoard graphs to work correctly. Simply call `tf.train.get_global_step` and pass the result to the global_step argument of minimize.

In [33]:
mode = tf.estimator.ModeKeys.TRAIN

optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

The `EstimatorSpec` returned for training must have the following fields set:

* `loss`, which contains the value of the loss function.
* `train_op`, which executes a training step.

In [31]:
tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

EstimatorSpec(mode='train', predictions={}, loss=<tf.Tensor 'sparse_softmax_cross_entropy_loss/value:0' shape=() dtype=float32>, train_op=<tf.Operation 'Adagrad' type=NoOp>, eval_metric_ops={}, export_outputs=None, training_chief_hooks=(), training_hooks=(), scaffold=<tensorflow.python.training.monitored_session.Scaffold object at 0x120ecc470>, evaluation_hooks=())

#### Putting it all together

In [35]:
def my_model(features, labels, mode, params):
    """DNN with three hidden layers, and dropout of 0.1 probability."""
    # Create three fully connected layers each layer having a dropout
    # probability of 0.1.
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


### Instantiate custom Estimator


In [38]:
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    params={
        'feature_columns': all_features,
        # Two hidden layers of 10 nodes each.
        'hidden_units': [10, 10],
        # The model must choose between 2 classes.
        'n_classes': 2,
    },
    model_dir="/tmp/tfcustom/")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_master': '', '_model_dir': '/tmp/tfcustom/', '_task_type': 'worker', '_session_config': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12121c358>, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_service': None}


In [39]:
classifier.train(
    steps=1000,
    input_fn=lambda : train_input_fn(features, labels, 32))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tfcustom/model.ckpt.
INFO:tensorflow:step = 1, loss = 4.754282
INFO:tensorflow:global_step/sec: 254.687
INFO:tensorflow:step = 101, loss = 0.52425635 (0.394 sec)
INFO:tensorflow:global_step/sec: 416.787
INFO:tensorflow:step = 201, loss = 0.63963854 (0.240 sec)
INFO:tensorflow:global_step/sec: 437.738
INFO:tensorflow:step = 301, loss = 0.6613085 (0.229 sec)
INFO:tensorflow:global_step/sec: 419.736
INFO:tensorflow:step = 401, loss = 0.48632666 (0.238 sec)
INFO:tensorflow:global_step/sec: 454.843
INFO:tensorflow:step = 501, loss = 0.41039792 (0.220 sec)
INFO:tensorflow:global_step/sec: 448.65
INFO:tensorflow:step = 601, loss = 0.43381172 (0.223 sec)
INFO:tensorflow:global_step/sec: 421.63
INFO:tensorflow:step = 701, loss = 0.5299153 (0.237 sec)
INFO:tensorflow:global_step/sec: 422.338
INFO:tensorflow:step = 801, loss = 0.41666624 (0.237 sec)
INFO:tensorflow:global_step/sec: 289.83
INFO:tensorflo

### Convolutional Estimator
Follow the guide [here| https://www.tensorflow.org/tutorials/layers]